In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import copy 

# Compute heat map (for pop. inversion or light emission)
def plot_heat(x, y, z, z_min, z_max):
    # x,y: x and y axis variables
    # z: dependent variable, dimension dim(x)*dim(y)
    # z_min, z_max: range of z to plot
    c = plt.pcolormesh(x, y, z, cmap = 'CMRmap', vmin = z_min, vmax = z_max)
    plt.colorbar(c)
    plt.title('$<\sigma_z>$',Fontsize=18)
    plt.ylabel('Detuning',Fontsize=18)
    plt.xlabel('Time$(\mu s)$',fontsize=18)

# Compute decay time
def findTd(sz,t_list):
  i=0
  if sz[-1]>0.0:
    return -1
  while i<len(t_list):
    if sz[i]>0.0:
        i+=1
    else:
        return t_list[i]

  

In [ ]:
%%writefile functions.cuh
#if !defined(FUNCTIONS_H)
#define FUNCTIONS_H

//all functions
#include <math.h>
#include <stdio.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>
// random number generation
#include <curand.h>
#include <curand_kernel.h>

using namespace std;



					
// update the observables of photons 
__global__ void	calculate_photons(double tc,int num_ens,double *para_a_dev,double *para_c_dev,\
				double2 *ap_a_dev,double2 *a_dev,double2 *a_a_dev,\
				double2 *a_sp_dev,double2 *sm_dev,double2 *a_sm_dev,\
				double2 *d_ap_a_dev,double2 *d_a_dev,double2 *d_a_a_dev);

__global__ void	update_photons(double t_step, double2 *ap_a_dev,double2 *a_dev,double2 *a_a_dev,\
				double2 *d_ap_a_dev,double2 *d_a_dev,double2 *d_a_a_dev);

// update the observables of atoms 
__global__ void	calculate_atoms(double tc,int num_ens,double *para_a_dev,double *para_c_dev,\
				double2 *sz_dev,double2 *sm_dev,double2 *a_sz_dev,double2 *a_sm_dev,double2 *a_sp_dev,\
				double2 *sm_sp_dev,double2 *sm_sm_dev,double2 *sm_sz_dev, double2 *a_dev, double2 *ap_a_dev,double2 *a_a_dev,\
				double2 *d_sz_dev,double2 *d_sm_dev,double2 *d_a_sz_dev,double2 *d_a_sm_dev,double2 *d_a_sp_dev);
							
							
__global__ void	update_atoms(int num_ens,double t_step,double *para_a_dev,double2 *sz_dev,double2 *sm_dev,double2 *a_sz_dev,double2 *a_sm_dev,double2 *a_sp_dev,\
				double2 *d_sz_dev,double2 *d_sm_dev,double2 *d_a_sz_dev,double2 *d_a_sm_dev,double2 *d_a_sp_dev);

							
__global__ void	calculate_correlations(int num_ens,double t_step,double *para_a_dev,double *para_c_dev,\
					double2 *sm_sp_dev,double2 *sm_sz_dev,double2 *sm_sm_dev,double2 *sz_sz_dev,\
					double2 *a_dev,double2 *a_sm_dev,double2 *a_sp_dev,double2 *a_sz_dev,double2 *sm_dev,double2 *sz_dev,\
					double2 *d_sm_sp_dev,double2 *d_sm_sz_dev,double2 *d_sm_sm_dev,double2 *d_sz_sz_dev);
							
							
__global__ void	update_correlations(int num_ens,double t_step,double2 *sm_sp_dev,double2 *sm_sz_dev,double2 *sm_sm_dev,double2 *sz_sz_dev,\
					double2 *d_sm_sp_dev,double2 *d_sm_sz_dev,double2 *d_sm_sm_dev,double2 *d_sz_sz_dev);

#endif

In [ ]:
%%writefile functions.cu
//all functions
#include <math.h>
#include <stdio.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>
// random number generation
#include <curand.h>
#include <curand_kernel.h>
// parameters and functions
#include "functions.cuh"

using namespace std;



// update the observables of photons 
__global__ void	calculate_photons(double tc,int num_ens,double *para_a_dev,double *para_c_dev,\
				double2 *ap_a_dev,double2 *a_dev,double2 *a_a_dev,\
				double2 *a_sp_dev,double2 *sm_dev,double2 *a_sm_dev,\
				double2 *d_ap_a_dev,double2 *d_a_dev,double2 *d_a_a_dev){
// laser pulse 
// if current time tc exceeds pulse duration then set F to 0
	if (tc > para_c_dev[5]){ 
		para_c_dev[4] = 0.;
	}

// photon number ada
// coupling 
	(*d_ap_a_dev).x = - (para_c_dev[1] + para_c_dev[2])*(*ap_a_dev).x;
	(*d_ap_a_dev).y = 0.;
	for (int i =0; i < num_ens; i++){
		(*d_ap_a_dev).x += -2.*para_a_dev[i+5*num_ens]*para_a_dev[i]*a_sp_dev[i].y;
	}
// driving 
	(*d_ap_a_dev).x += -2.*sqrtf(para_c_dev[1])*para_c_dev[4]*(cos(para_c_dev[3]*tc)*(*a_dev).y + sin(para_c_dev[3]*tc)*(*a_dev).x);


	
//  photon amplitude a
	double2 c_omega_c;
	c_omega_c.x = para_c_dev[0];
	c_omega_c.y = -0.5*(para_c_dev[1] + para_c_dev[2]);

	(*d_a_dev).x = c_omega_c.y*(*a_dev).x+c_omega_c.x*(*a_dev).y;
	(*d_a_dev).y = -(c_omega_c.x*(*a_dev).x -c_omega_c.y*(*a_dev).y); 
	
	for (int i = 0; i < num_ens; i++){
		(*d_a_dev).x += para_a_dev[i+5*num_ens]*para_a_dev[i]*sm_dev[i].y;
		(*d_a_dev).y += - para_a_dev[i+5*num_ens]*para_a_dev[i]*sm_dev[i].x;
	}	
// driving 	
	(*d_a_dev).x += sqrtf(para_c_dev[1])*para_c_dev[4]*sin(-para_c_dev[3]*tc); 
	(*d_a_dev).y += - sqrtf(para_c_dev[1])*para_c_dev[4]*cos(-para_c_dev[3]*tc);



//
// photon-photon correlation a2
	(*d_a_a_dev).x = 2.*(c_omega_c.x*(*a_a_dev).y + c_omega_c.y*(*a_a_dev).x); 
	(*d_a_a_dev).y = -2.*(c_omega_c.x*(*a_a_dev).x - c_omega_c.y*(*a_a_dev).y);
	
	for (int i = 0; i< num_ens; i++) {
		(*d_a_a_dev).x += 2.*para_a_dev[i+5*num_ens]*para_a_dev[i]*a_sm_dev[i].y;
		(*d_a_a_dev).y += - 2.*para_a_dev[i+5*num_ens]*para_a_dev[i]*a_sm_dev[i].x;
	}
// driving 	
	(*d_a_a_dev).x += 2.*sqrtf(para_c_dev[1])*para_c_dev[4]*(cos(-para_c_dev[3]*tc)*(*a_dev).y + sin(-para_c_dev[3]*tc)*(*a_dev).x); 
	(*d_a_a_dev).y += -2.*sqrtf(para_c_dev[1])*para_c_dev[4]*(cos(-para_c_dev[3]*tc)*(*a_dev).x - sin(-para_c_dev[3]*tc)*(*a_dev).y);
										  
}


__global__ void	update_photons(double t_step, double2 *ap_a_dev,double2 *a_dev,double2 *a_a_dev,\
								double2 *d_ap_a_dev,double2 *d_a_dev,double2 *d_a_a_dev){
	(*ap_a_dev).x += t_step*(*d_ap_a_dev).x;
	
	(*a_dev).x += t_step*(*d_a_dev).x;
	(*a_dev).y += t_step*(*d_a_dev).y;
	
	(*a_a_dev).x += t_step*(*d_a_a_dev).x;
	(*a_a_dev).y += t_step*(*d_a_a_dev).y;
}


// update the observables of atoms 
__global__ void	calculate_atoms(double tc,int num_ens,double *para_a_dev,double *para_c_dev,\
				double2 *sz_dev,double2 *sm_dev,double2 *a_sz_dev,double2 *a_sm_dev,double2 *a_sp_dev,\
				double2 *sm_sp_dev,double2 *sm_sm_dev,double2 *sm_sz_dev, double2 *a_dev, double2 *ap_a_dev, double2 *a_a_dev,\
				double2 *d_sz_dev,double2 *d_sm_dev,double2 *d_a_sz_dev,double2 *d_a_sm_dev,double2 *d_a_sp_dev){
// calculate the index of matrix
int i = threadIdx.x + blockIdx.x*blockDim.x;
if (i < num_ens){
// atomic population inversion 
		d_sz_dev[i].x = 4.*para_a_dev[i+5*num_ens]*a_sp_dev[i].y\
				- para_a_dev[i+2*num_ens]*(1.+sz_dev[i].x) + para_a_dev[i+3*num_ens]*(1.-sz_dev[i].x);
		d_sz_dev[i].y = 0.;
	
// atomic polarization
		double2 c_omega_a;
		c_omega_a.x = para_a_dev[i+num_ens];
		c_omega_a.y = -(0.5*(para_a_dev[i+2*num_ens]+para_a_dev[i+3*num_ens]) + para_a_dev[i+4*num_ens]);
		
		d_sm_dev[i].x  = c_omega_a.x*sm_dev[i].y + c_omega_a.y*sm_dev[i].x - para_a_dev[i+5*num_ens]*a_sz_dev[i].y;
		d_sm_dev[i].y  = -(c_omega_a.x*sm_dev[i].x - c_omega_a.y*sm_dev[i].y) + para_a_dev[i+5*num_ens]*a_sz_dev[i].x;
		
// atom-photon correlation
// a_sp 
		double2 c_omega_c;
		c_omega_c.x = 0.;
		c_omega_c.y = -0.5*(para_c_dev[1] + para_c_dev[2]);

		double a_ap_sz = 2.*(a_sz_dev[i].x*(*a_dev).x + a_sz_dev[i].y*(*a_dev).y) \
				+ (1. + (*ap_a_dev).x)*sz_dev[i].x - 2.*((*a_dev).x*(*a_dev).x + (*a_dev).y*(*a_dev).y)*sz_dev[i].x;
						
		d_a_sp_dev[i].x = -((c_omega_a.x-c_omega_c.x)*a_sp_dev[i].y+(-c_omega_a.y-c_omega_c.y)*a_sp_dev[i].x) \
					+ para_a_dev[i+5*num_ens]*(para_a_dev[i]-1.)*sm_sp_dev[i+i*num_ens].y;
						
		d_a_sp_dev[i].y = (c_omega_a.x-c_omega_c.x)*a_sp_dev[i].x - (-c_omega_a.y-c_omega_c.y)*a_sp_dev[i].y\
					- 0.5*para_a_dev[i+5*num_ens]*(1.-sz_dev[i].x) \
					- para_a_dev[i+5*num_ens]*(para_a_dev[i]-1.)*sm_sp_dev[i+i*num_ens].x \
					- para_a_dev[i+5*num_ens]*a_ap_sz;
			
		for (int j = 0; j < num_ens; j ++){
			if (j != i){
				d_a_sp_dev[i].x += para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sp_dev[j+i*num_ens].y;
				d_a_sp_dev[i].y += - para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sp_dev[j+i*num_ens].x;
			}
		}

// driving 	
		d_a_sp_dev[i].x += sqrtf(para_c_dev[1])*para_c_dev[4]*(-cos(-para_c_dev[3]*tc)*sm_dev[i].y + sin(-para_c_dev[3]*tc)*sm_dev[i].x);
		d_a_sp_dev[i].y += - sqrtf(para_c_dev[1])*para_c_dev[4]*(cos(-para_c_dev[3]*tc)*sm_dev[i].x + sin(-para_c_dev[3]*tc)*sm_dev[i].y);

					
// a_sm
		double2 a_a_sz;
		a_a_sz.x = 2.*((*a_dev).x*a_sz_dev[i].x - (*a_dev).y*a_sz_dev[i].y) \
				+ (*a_a_dev).x*sz_dev[i].x - 2.*((*a_dev).x*(*a_dev).x -(*a_dev).y*(*a_dev).y)*sz_dev[i].x;
		a_a_sz.y = 2.*((*a_dev).y*a_sz_dev[i].x + (*a_dev).x*a_sz_dev[i].y) \
				+ (*a_a_dev).y*sz_dev[i].x - 2.*(2.*(*a_dev).x*(*a_dev).y)*sz_dev[i].x;

		d_a_sm_dev[i].x = (c_omega_c.x + c_omega_a.x)*a_sm_dev[i].y +  (c_omega_c.y + c_omega_a.y)*a_sm_dev[i].x\
					+ para_a_dev[i+5*num_ens]*((para_a_dev[i]-1.)*sm_sm_dev[i+i*num_ens].y - a_a_sz.y);
		
		d_a_sm_dev[i].y = -((c_omega_c.x + c_omega_a.x)*a_sm_dev[i].x- (c_omega_c.y + c_omega_a.y)*a_sm_dev[i].y) \
					- para_a_dev[i+5*num_ens]*((para_a_dev[i]-1.)*sm_sm_dev[i+i*num_ens].x - a_a_sz.x); 		
		
		for (int j = 0; j < num_ens; j ++){
			if (j != i){
				d_a_sm_dev[i].x += para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sm_dev[j+i*num_ens].y;
				d_a_sm_dev[i].y += -para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sm_dev[j+i*num_ens].x;
			}
		}

// driving 
		d_a_sm_dev[i].x += sqrtf(para_c_dev[1])*para_c_dev[4]*(sin(-para_c_dev[3]*tc)*sm_dev[i].x + cos(-para_c_dev[3]*tc)*sm_dev[i].y);
		d_a_sm_dev[i].y += - sqrtf(para_c_dev[1])*para_c_dev[4]*(cos(-para_c_dev[3]*tc)*sm_dev[i].x - sin(-para_c_dev[3]*tc)*sm_dev[i].y);


// a_sz
		double2 a_ap_sm,a_a_sp;

		a_a_sp.x = (*a_a_dev).x*sm_dev[i].x + (*a_a_dev).y*sm_dev[i].y + 2.*((*a_dev).x*a_sp_dev[i].x- (*a_dev).y*a_sp_dev[i].y) \
			-2.*(((*a_dev).x*(*a_dev).x - (*a_dev).y*(*a_dev).y)*sm_dev[i].x  + 2.*(*a_dev).x*(*a_dev).y*sm_dev[i].y);
		a_a_sp.y = -(*a_a_dev).x*sm_dev[i].y + (*a_a_dev).y*sm_dev[i].x + 2.*((*a_dev).x*a_sp_dev[i].y + (*a_dev).y*a_sp_dev[i].x)\
			-2.*(-((*a_dev).x*(*a_dev).x - (*a_dev).y*(*a_dev).y)*sm_dev[i].y + 2.*(*a_dev).x*(*a_dev).y*sm_dev[i].x);

		a_ap_sm.x = a_sm_dev[i].x*(*a_dev).x + a_sm_dev[i].y*(*a_dev).y + (1. + (*ap_a_dev).x)*sm_dev[i].x \
			+ (*a_dev).x*a_sp_dev[i].x + (*a_dev).y*a_sp_dev[i].y - 2.*((*a_dev).x*(*a_dev).x + (*a_dev).y*(*a_dev).y)*sm_dev[i].x;
		a_ap_sm.y = a_sm_dev[i].y*(*a_dev).x - a_sm_dev[i].x*(*a_dev).y + (1. + (*ap_a_dev).x)*sm_dev[i].y \
			+ (*a_dev).y*a_sp_dev[i].x - (*a_dev).x*a_sp_dev[i].y - 2.*((*a_dev).x*(*a_dev).x + (*a_dev).y*(*a_dev).y)*sm_dev[i].y;
	
		d_a_sz_dev[i].x = c_omega_c.x*a_sz_dev[i].y + c_omega_c.y*a_sz_dev[i].x \
				- para_a_dev[i+2*num_ens]*((*a_dev).x +a_sz_dev[i].x) + para_a_dev[i+3*num_ens]*((*a_dev).x-a_sz_dev[i].x)\
				+ para_a_dev[i+5*num_ens]*(sm_dev[i].y + (para_a_dev[i]-1.)*sm_sz_dev[i+i*num_ens].y) \
				+ 2.*para_a_dev[i+5*num_ens]*(a_a_sp.y - a_ap_sm.y);
				
		d_a_sz_dev[i].y = -(c_omega_c.x*a_sz_dev[i].x - c_omega_c.y*a_sz_dev[i].y) \
				- para_a_dev[i+2*num_ens]*((*a_dev).y + a_sz_dev[i].y) + para_a_dev[i+3*num_ens]*((*a_dev).y - a_sz_dev[i].y)\
				- para_a_dev[i+5*num_ens]*(sm_dev[i].x + (para_a_dev[i]-1.)*sm_sz_dev[i+i*num_ens].x) \
				- 2.*para_a_dev[i+5*num_ens]*(a_a_sp.x - a_ap_sm.x);
				

		for (int j = 0; j < num_ens; j ++){
			if (j != i){
				d_a_sz_dev[i].x += para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sz_dev[j+i*num_ens].y;
				d_a_sz_dev[i].y += -para_a_dev[j+5*num_ens]*para_a_dev[j]*sm_sz_dev[j+i*num_ens].x;
			}
		}
// driving 		
		d_a_sz_dev[i].x += sqrtf(para_c_dev[1])*para_c_dev[4]*sin(-para_c_dev[3]*tc)*sz_dev[i].x;
		d_a_sz_dev[i].y += -sqrtf(para_c_dev[1])*para_c_dev[4]*cos(-para_c_dev[3]*tc)*sz_dev[i].x;

}							
}

__global__ void	update_atoms(int num_ens,double t_step,double *para_a_dev,double2 *sz_dev,double2 *sm_dev,double2 *a_sz_dev,double2 *a_sm_dev,double2 *a_sp_dev,\
				double2 *d_sz_dev,double2 *d_sm_dev,double2 *d_a_sz_dev,double2 *d_a_sm_dev,double2 *d_a_sp_dev){
// calculate the index of matrix
int i = threadIdx.x + blockIdx.x*blockDim.x;
if (i < num_ens){
// atom loss
	para_a_dev[i] += -t_step*para_a_dev[i + 6*num_ens]*para_a_dev[i];
	
	sz_dev[i].x += t_step*d_sz_dev[i].x;
	
	sm_dev[i].x += t_step*d_sm_dev[i].x;
	sm_dev[i].y += t_step*d_sm_dev[i].y;
	
	a_sz_dev[i].x += t_step*d_a_sz_dev[i].x;
	a_sz_dev[i].y += t_step*d_a_sz_dev[i].y;
	
	a_sm_dev[i].x += t_step*d_a_sm_dev[i].x;
	a_sm_dev[i].y += t_step*d_a_sm_dev[i].y;
	
	a_sp_dev[i].x += t_step*d_a_sp_dev[i].x;
	a_sp_dev[i].y += t_step*d_a_sp_dev[i].y;
}
}



__global__ void	calculate_correlations(int num_ens,double t_step,double *para_a_dev,double *para_c_dev,\
					double2 *sm_sp_dev,double2 *sm_sz_dev,double2 *sm_sm_dev,double2 *sz_sz_dev,\
					double2 *a_dev,double2 *a_sm_dev,double2 *a_sp_dev,double2 *a_sz_dev,double2 *sm_dev,double2 *sz_dev,\
					double2 *d_sm_sp_dev,double2 *d_sm_sz_dev,double2 *d_sm_sm_dev,double2 *d_sz_sz_dev){
int i = threadIdx.x;
int j = blockIdx.x;
if (i < num_ens){
	if (j < num_ens) {
//
// atom-atom correlation
		double2 c_omega_a_i,c_omega_a_j;
		c_omega_a_i.x = para_a_dev[i+num_ens];
		c_omega_a_i.y = -(0.5*(para_a_dev[i+2*num_ens]+para_a_dev[i+3*num_ens]) + para_a_dev[i+4*num_ens]);
		c_omega_a_j.x = para_a_dev[j+num_ens];
		c_omega_a_j.y = -(0.5*(para_a_dev[j+2*num_ens]+para_a_dev[j+3*num_ens]) + para_a_dev[j+4*num_ens]);

// sm_sp
		double2 a_sz_sp,ap_sm_sz;
		a_sz_sp.x = (*a_dev).x*sm_sz_dev[i + j*num_ens].x + (*a_dev).y*sm_sz_dev[i + j*num_ens].y + a_sz_dev[j].x*sm_dev[i].x + a_sz_dev[j].y*sm_dev[i].y \
			+ sz_dev[j].x*a_sp_dev[i].x - 2.*sz_dev[j].x*((*a_dev).x*sm_dev[i].x + (*a_dev).y*sm_dev[i].y);
		a_sz_sp.y = (*a_dev).y*sm_sz_dev[i + j*num_ens].x - (*a_dev).x*sm_sz_dev[i + j*num_ens].y + a_sz_dev[j].y*sm_dev[i].x - a_sz_dev[j].x*sm_dev[i].y \
			+ sz_dev[j].x*a_sp_dev[i].y - 2.*sz_dev[j].x*((*a_dev).y*sm_dev[i].x - (*a_dev).x*sm_dev[i].y);
		
		ap_sm_sz.x = (*a_dev).x*sm_sz_dev[j + i*num_ens].x + (*a_dev).y*sm_sz_dev[j + i*num_ens].y + sm_dev[j].x*a_sz_dev[i].x + sm_dev[j].y*a_sz_dev[i].y\
			+ a_sp_dev[j].x*sz_dev[i].x -2.*((*a_dev).x*sm_dev[j].x + (*a_dev).y*sm_dev[j].y)*sz_dev[i].x;
		ap_sm_sz.y = -(*a_dev).y*sm_sz_dev[j + i*num_ens].x + (*a_dev).x*sm_sz_dev[j + i*num_ens].y + sm_dev[j].y*a_sz_dev[i].x - sm_dev[j].x*a_sz_dev[i].y\
			- a_sp_dev[j].y*sz_dev[i].x -2.*(-(*a_dev).y*sm_dev[j].x + (*a_dev).x*sm_dev[j].y)*sz_dev[i].x;

		d_sm_sp_dev[j + i*num_ens].x = ((c_omega_a_j.x - c_omega_a_i.x)*sm_sp_dev[j + i*num_ens].y + (c_omega_a_j.y + c_omega_a_i.y)*sm_sp_dev[j + i*num_ens].x) \
						- para_a_dev[j+5*num_ens]*a_sz_sp.y + para_a_dev[i+5*num_ens]*ap_sm_sz.y;
		d_sm_sp_dev[j + i*num_ens].y = - ((c_omega_a_j.x - c_omega_a_i.x)*sm_sp_dev[j + i*num_ens].x - (c_omega_a_j.y + c_omega_a_i.y)*sm_sp_dev[j + i*num_ens].y) \
						+ para_a_dev[j+5*num_ens]*a_sz_sp.x - para_a_dev[i+5*num_ens]*ap_sm_sz.x;
																
// sm_sz
		double2 a_sz_sz,a_sm_sp,ap_sm_sm;
		a_sz_sz.x = a_sz_dev[j].x*sz_dev[i].x + (*a_dev).x*sz_sz_dev[j + i*num_ens].x +sz_dev[j].x*a_sz_dev[i].x -2.*(*a_dev).x*sz_dev[j].x*sz_dev[i].x;
		a_sz_sz.y = a_sz_dev[j].y*sz_dev[i].x + (*a_dev).y*sz_sz_dev[j + i*num_ens].x +sz_dev[j].x*a_sz_dev[i].y -2.*(*a_dev).y*sz_dev[j].x*sz_dev[i].x;
		
		a_sm_sp.x = (*a_dev).x*sm_sp_dev[j + i*num_ens].x - (*a_dev).y*sm_sp_dev[j + i*num_ens].y + sm_dev[j].x*a_sp_dev[i].x - sm_dev[j].y*a_sp_dev[i].y\
			+ a_sm_dev[j].x*sm_dev[i].x + a_sm_dev[j].y*sm_dev[i].y\
			-2.*((*a_dev).x*(sm_dev[j].x*sm_dev[i].x + sm_dev[j].y*sm_dev[i].y) - (*a_dev).y*(sm_dev[j].y*sm_dev[i].x - sm_dev[j].x*sm_dev[i].y));
		a_sm_sp.y = (*a_dev).y*sm_sp_dev[j + i*num_ens].x + (*a_dev).x*sm_sp_dev[j + i*num_ens].y + sm_dev[j].y*a_sp_dev[i].x + sm_dev[j].x*a_sp_dev[i].y \
			+ a_sm_dev[j].y*sm_dev[i].x - a_sm_dev[j].x*sm_dev[i].y  \
			-2.*((*a_dev).y*(sm_dev[j].x*sm_dev[i].x + sm_dev[j].y*sm_dev[i].y) + (*a_dev).x*(sm_dev[j].y*sm_dev[i].x - sm_dev[j].x*sm_dev[i].y));
		
		ap_sm_sm.x = (*a_dev).x*sm_sm_dev[j + i*num_ens].x + (*a_dev).y*sm_sm_dev[j + i*num_ens].y + sm_dev[j].x*a_sp_dev[i].x  + sm_dev[j].y*a_sp_dev[i].y \
			+ a_sp_dev[j].x*sm_dev[i].x + a_sp_dev[j].y*sm_dev[i].y \
			-2.*((*a_dev).x*(sm_dev[j].x*sm_dev[i].x - sm_dev[j].y*sm_dev[i].y) + (*a_dev).y*(sm_dev[j].x*sm_dev[i].y + sm_dev[j].y*sm_dev[i].x));
		ap_sm_sm.y = -(*a_dev).y*sm_sm_dev[j + i*num_ens].x + (*a_dev).x*sm_sm_dev[j + i*num_ens].y + sm_dev[j].y*a_sp_dev[i].x  - sm_dev[j].x*a_sp_dev[i].y \
			- a_sp_dev[j].y*sm_dev[i].x + a_sp_dev[j].x*sm_dev[i].y \
			-2.*(-(*a_dev).y*(sm_dev[j].x*sm_dev[i].x - sm_dev[j].y*sm_dev[i].y) + (*a_dev).x*(sm_dev[j].x*sm_dev[i].y + sm_dev[j].y*sm_dev[i].x));

		d_sm_sz_dev[j + i*num_ens].x = c_omega_a_j.x*sm_sz_dev[j + i*num_ens].y +  c_omega_a_j.y*sm_sz_dev[j + i*num_ens].x\
				- para_a_dev[j+5*num_ens]*a_sz_sz.y + 2.*para_a_dev[i+5*num_ens]*(a_sm_sp.y - ap_sm_sm.y)\
				- para_a_dev[i+2*num_ens]*(sm_dev[j].x + sm_sz_dev[j + i*num_ens].x) + para_a_dev[i+3*num_ens]*(sm_dev[j].x - sm_sz_dev[j + i*num_ens].x);													
		d_sm_sz_dev[j + i*num_ens].y = -(c_omega_a_j.x*sm_sz_dev[j + i*num_ens].x - c_omega_a_j.y*sm_sz_dev[j + i*num_ens].y)\
				+ para_a_dev[j+5*num_ens]*a_sz_sz.x - 2.*para_a_dev[i+5*num_ens]*(a_sm_sp.x - ap_sm_sm.x)\
				- para_a_dev[i+2*num_ens]*(sm_dev[j].y+sm_sz_dev[j + i*num_ens].y) + para_a_dev[i+3*num_ens]*(sm_dev[j].y - sm_sz_dev[j + i*num_ens].y);
// sm_sm
		double2 a_sm_sz,a_sz_sm;
		a_sm_sz.x = (*a_dev).x*sm_sz_dev[j + i*num_ens].x - (*a_dev).y*sm_sz_dev[j + i*num_ens].y + sm_dev[j].x*a_sz_dev[i].x - sm_dev[j].y*a_sz_dev[i].y\
				  + a_sm_dev[j].x*sz_dev[i].x -2.*((*a_dev).x*sm_dev[j].x - (*a_dev).y*sm_dev[j].y)*sz_dev[i].x;
		a_sm_sz.y = (*a_dev).y*sm_sz_dev[j + i*num_ens].x + (*a_dev).x*sm_sz_dev[j + i*num_ens].y + sm_dev[j].y*a_sz_dev[i].x + sm_dev[j].x*a_sz_dev[i].y\
				  + a_sm_dev[j].y*sz_dev[i].x -2.*((*a_dev).y*sm_dev[j].x + (*a_dev).x*sm_dev[j].y)*sz_dev[i].x;
				  
		a_sz_sm.x = (*a_dev).x*sm_sz_dev[i + j*num_ens].x - (*a_dev).y*sm_sz_dev[i + j*num_ens].y + a_sz_dev[j].x*sm_dev[i].x - a_sz_dev[j].y*sm_dev[i].y\
				  + sz_dev[j].x*a_sm_dev[i].x -2.*sz_dev[j].x*((*a_dev).x*sm_dev[i].x - (*a_dev).y*sm_dev[i].y);
		a_sz_sm.y = (*a_dev).y*sm_sz_dev[i + j*num_ens].x + (*a_dev).x*sm_sz_dev[i + j*num_ens].y + a_sz_dev[j].y*sm_dev[i].x + a_sz_dev[j].x*sm_dev[i].y\
				  + sz_dev[j].x*a_sm_dev[i].y -2.*sz_dev[j].x*((*a_dev).y*sm_dev[i].x + (*a_dev).x*sm_dev[i].y);
		
		d_sm_sm_dev[j + i*num_ens].x = ((c_omega_a_j.y + c_omega_a_i.y)*sm_sm_dev[j + i*num_ens].x + (c_omega_a_j.x + c_omega_a_i.x)*sm_sm_dev[j + i*num_ens].y) \
					- para_a_dev[i+5*num_ens]*a_sm_sz.y - para_a_dev[j+5*num_ens]*a_sz_sm.y;
		d_sm_sm_dev[j + i*num_ens].y = - ((c_omega_a_j.x + c_omega_a_i.x)*sm_sm_dev[j + i*num_ens].x - (c_omega_a_j.y + c_omega_a_i.y)*sm_sm_dev[j + i*num_ens].y) \
					+ para_a_dev[i+5*num_ens]*a_sm_sz.x + para_a_dev[j+5*num_ens]*a_sz_sm.x;

// sz_sz							
		double2 a_sp_sz;
		
		a_sp_sz.x = (*a_dev).x*sm_sz_dev[j + i*num_ens].x + (*a_dev).y*sm_sz_dev[j + i*num_ens].y + sm_dev[j].x*a_sz_dev[i].x + sm_dev[j].y*a_sz_dev[i].y \
			+ a_sp_dev[j].x*sz_dev[i].x - 2.*sz_dev[i].x*((*a_dev).x*sm_dev[j].x + (*a_dev).y*sm_dev[j].y);
		a_sp_sz.y = (*a_dev).y*sm_sz_dev[j + i*num_ens].x - (*a_dev).x*sm_sz_dev[j + i*num_ens].y - sm_dev[j].y*a_sz_dev[i].x + sm_dev[j].x*a_sz_dev[i].y \
			+ a_sp_dev[j].y*sz_dev[i].x - 2.*sz_dev[i].x*((*a_dev).y*sm_dev[j].x - (*a_dev).x*sm_dev[j].y);

		d_sz_sz_dev[j + i*num_ens].x = 4.*para_a_dev[i+5*num_ens]*a_sz_sp.y + 4.*para_a_dev[j+5*num_ens]*a_sp_sz.y \
					- para_a_dev[i+2*num_ens]*(sz_dev[j].x + sz_sz_dev[j + i*num_ens].x) + para_a_dev[i+3*num_ens]*(sz_dev[j].x-sz_sz_dev[j + i*num_ens].x) \
					- para_a_dev[j+2*num_ens]*(sz_dev[i].x + sz_sz_dev[j + i*num_ens].x) + para_a_dev[j+3*num_ens]*(sz_dev[i].x-sz_sz_dev[j + i*num_ens].x);
		d_sz_sz_dev[j + i*num_ens].y = 0.;		
	}
}															
}

__global__ void	update_correlations(int num_ens,double t_step,double2 *sm_sp_dev,double2 *sm_sz_dev,double2 *sm_sm_dev,double2 *sz_sz_dev,\
				double2 *d_sm_sp_dev,double2 *d_sm_sz_dev,double2 *d_sm_sm_dev,double2 *d_sz_sz_dev){
int i = threadIdx.x;
int j = blockIdx.x;
if (i < num_ens){
	if (j < num_ens){
		sm_sp_dev[j + i*num_ens].x += t_step*d_sm_sp_dev[j + i*num_ens].x; 
		sm_sp_dev[j + i*num_ens].y += t_step*d_sm_sp_dev[j + i*num_ens].y; 
		
		sm_sz_dev[j + i*num_ens].x += t_step*d_sm_sz_dev[j + i*num_ens].x;
		sm_sz_dev[j + i*num_ens].y += t_step*d_sm_sz_dev[j + i*num_ens].y; 
		
		sm_sm_dev[j + i*num_ens].x += t_step*d_sm_sm_dev[j + i*num_ens].x;
		sm_sm_dev[j + i*num_ens].y += t_step*d_sm_sm_dev[j + i*num_ens].y; 
		
		sz_sz_dev[j + i*num_ens].x += t_step*d_sz_sz_dev[j + i*num_ens].x;
		sz_sz_dev[j + i*num_ens].y += t_step*d_sz_sz_dev[j + i*num_ens].y; 
	}
}
}

In [ ]:
%%writefile main.cu
#include <math.h>
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>
#include <ctime>
#include <iostream>
using namespace std;
#include <curand_kernel.h>
#include <curand.h>
#include "functions.cuh"
#define PI (4.0 * atan(1.0));
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}


int main(int argc, char** argv) {

int num_ens = atoi(argv[1]); // number of classes
int p=50; // need this for symmetric freq dist // num_ens=2p+1
int N_total = atoi(argv[2]); // number of spins


clock_t ct0,ct1;
ct0 = clock();
double axe[p+1];
int ens_size = N_total/num_ens;
double phi_0 = atof(argv[4])*PI;
double inhomo[num_ens];



double b= 0.999999;
double aa= 0.0000000001;
int n= p+1;
double c = (b - aa)/(n - 1);
for(int i = 0; i < n; ++i){
axe[i] =  aa + i*c;
//printf("%f \n",axe[i]);
}
axe[n - 1] = b;

double maxdetun = 500;
double sigma = 0.022;
double sqrthalf = 0.707;



// Normal distribution
// Writing detuning values in a file 
FILE *Detuning;
Detuning = fopen("Detuning.dat","w");

if(num_ens==1){
	double freq = 0;
inhomo[0] = freq; //always start at zero detuning
fprintf(Detuning,"%e \n",inhomo[0]);
fclose(Detuning);
}

//if(num_ens>1){
//	for (int i=1; i<p+1; i++){
//		double x= axe[i];
//		double tt1, tt2, lnx, sgn;
//		sgn = (x < 0) ? -1.0f : 1.0f;
//		x = (1 - x)*(1 + x);    
//		lnx = log(x);
//		tt1 = (2/(3.1416*0.147) + 0.5f * lnx);
//		tt2 = 1/(0.147) * lnx;
//		//inhomo[p+i] =  sgn*sqrt(-tt1 + sqrtf(tt1*tt1 - tt2))*maxdetun/3.1416;
//		//inhomo[p-i] = -sgn*sqrt(-tt1 + sqrtf(tt1*tt1 - tt2))*maxdetun/3.1416; 
//		inhomo[i-1] = ((0.5 * erfc(-axe[i] * sqrthalf/ sigma) - 0.5)*2-1)*maxdetun;
//		inhomo[2*p+1-i] = -((0.5 * erfc(-axe[i] * sqrthalf/ sigma) - 0.5)*2-1)*maxdetun;
//		//printf("%1f\n", inhomo[i+1]);
//	}
//	inhomo[p]=0;
//	for(int i=0;i<num_ens;i++){
//		fprintf(Detuning,"%e \n",inhomo[i]);
//	}
//	fclose(Detuning);
//}
if(num_ens==5){
double delta = atof(argv[3]);
double delta1 = atof(argv[4]);
int i0 = 0; int i1 = 1; int i2=2; int i3=3; int i4=4;
inhomo[i0]= -delta1;
fprintf(Detuning,"%e \n",inhomo[0]);
inhomo[i1]= -delta;
fprintf(Detuning,"%e \n",inhomo[1]);
inhomo[i2]= delta-delta;
fprintf(Detuning,"%e \n",inhomo[2]);
inhomo[i3]= delta;
fprintf(Detuning,"%e \n",inhomo[3]);
inhomo[i4]= delta1;
fprintf(Detuning,"%e \n",inhomo[4]);
fclose(Detuning);
}

if(num_ens==3){
double delta = atof(argv[3]);
int i0 = 0; int i1 = 1; int i2=2;
inhomo[i0]= -delta;
fprintf(Detuning,"%e \n",inhomo[0]);
inhomo[i1]= delta-delta;
fprintf(Detuning,"%e \n",inhomo[1]);
inhomo[i2]= delta;
fprintf(Detuning,"%e \n",inhomo[2]);
fclose(Detuning);
}

if(num_ens==2){
double delta = atof(argv[3]);
int i0 = 0; int i1 = 1;
inhomo[i0]= -delta;
fprintf(Detuning,"%e \n",inhomo[0]);
inhomo[i1]= delta-delta;
fprintf(Detuning,"%e \n",inhomo[1]);
fclose(Detuning);
}



//********************************************************************************************* PARAMETERS AND DEFINITIONS ********************************************************
// Unit in kHz * 2pi
double gamma_a_0 = 0.0*2.0;				// ATOM DECAY
double eta_a_0 =   0.0;						// ATOM PUMPING
double chi_a_0 =   0.0;						// ATOM DEPHASING
double coup_a_0 =  atof(argv[9]);					// ATOM CAVITY COUPLING
double loss_0 =    0.0;						// ATOM LOSS
double kappa_1_c = 1.0*100.0;				// LEFT MIRROR DECAY
double kappa_2_c = 1.0*100.0;				// RIGHT MIRROR DECAY

double theta_0 = atof(argv[6])*PI; 						// INTIAL STATE...... PI FOR EXCITED STATE AND (PI*0.5) FOR EQUAL SUPERPOSITION
double zero_phonon= 0.0;					// ZERO PHONON LINE
double omega_c = zero_phonon;					// CAVITY FREQUENCY AT RESONANCE




//********************************************************************************************* PARAMETERS FOR SQUARE PULSE ********************************************************

double omega_d = 0.2;						// FREQUENCY OF SQUARE PULSE FOR INITIALIZATION
//double coup_d =  0.0;					        // AMPLITUDE OF THE PULSE
double coup_d = 0.0*3;					// AMPLITUDE OF THE PULSE
double t_stop = 0.0*15; 					// LENGTH OF SQUARE PULSE in us
//1.943*1.0E-7			

//********************************************************************************************* PARAMETERS FOR TIME EVOLUTION *****************************************************

double t_max = atof(argv[7]);					// T_END
int t_num = atoi(argv[8]);						// NUMBER OF STEPS
double t_step = t_max/t_num;					// dT (SIZE OF EACH STEP)



//********************************************************************************************* PARAMETERS FOR OUTPUT POINTS *****************************************************

int t_store_num = 20000;
int t_store =  t_num/t_store_num;

//***************************************************************************************** INITIALIZATION & PROGRAM BEGINS HERE **************************************************



double N_a[num_ens],omega_a[num_ens],gamma_a[num_ens],\
		eta_a[num_ens],chi_a[num_ens],coup_a[num_ens],loss_a[num_ens];


for (int i =0; i < num_ens; i++){
	N_a[i] = ens_size;
	omega_a[i] = inhomo[i];
	gamma_a[i] = gamma_a_0;
	eta_a[i] = eta_a_0;
	chi_a[i] = chi_a_0;
	coup_a[i] = coup_a_0;
	loss_a[i] = loss_0;
}


// the parameters in an array 
double para_a[7*num_ens];
for  (int i = 0; i < num_ens; i++){
	para_a[i] = N_a[i];
	para_a[i+num_ens] = omega_a[i];
	para_a[i+2*num_ens] = gamma_a[i];
	para_a[i+3*num_ens] = eta_a[i];
	para_a[i+4*num_ens] = chi_a[i];
	para_a[i+5*num_ens] = coup_a[i];
	para_a[i+6*num_ens] = loss_a[i];
}


// copy the parameters into the memory in GPU
double *para_a_dev;
cudaMalloc((void**)&para_a_dev,6*num_ens*sizeof(double)); 
cudaMemcpy(para_a_dev,para_a,6*num_ens*sizeof(double),cudaMemcpyHostToDevice);

//*******************************
// parameters for initial states 


double theta[num_ens],phi[num_ens];
for (int i=0; i < num_ens; i++){
	theta[i] = theta_0;
	phi[i] = phi_0;
}

double2 cu[num_ens],cl[num_ens];
for (int i=0; i< num_ens; i++){
	cu[i].x = sin(0.5*theta[i])*cos(phi[i]);
	cu[i].y = sin(0.5*theta[i])*sin(phi[i]);
	
	cl[i].x = cos(0.5*theta[i]); 
	cl[i].y = 0.; 
}




double para_c[9];
para_c[0] = omega_c;
para_c[1] = kappa_1_c;
para_c[2] = kappa_2_c;

para_c[3] = omega_d;
para_c[4] = coup_d;
para_c[5] = t_stop;



double *para_c_dev;
cudaMalloc((void**)&para_c_dev,9*sizeof(double));
cudaMemcpy(para_c_dev,para_c,9*sizeof(double),cudaMemcpyHostToDevice);


double *t_step_dev;
cudaMalloc((void**)&t_step_dev,sizeof(double));
cudaMemcpy(t_step_dev,&t_step,sizeof(double),cudaMemcpyHostToDevice);







// on CPU side 
double2 ap_a,a,a_a;
double2 sz[num_ens],sm[num_ens],a_sz[num_ens],a_sm[num_ens],a_sp[num_ens];
double2 sm_sp[num_ens*num_ens],sm_sz[num_ens*num_ens],\
	sm_sm[num_ens*num_ens],sz_sz[num_ens*num_ens];

// for initial values 
double2 sm_1,sp_1,sz_1,sm_2,sz_2; 

//****************************
// initialize the observables
ap_a.x = 0.; ap_a.y = 0.; a.x = 0.; a.y = 0.; a_a.x =0.; a_a.y = 0.; 

for (int i= 0; i < num_ens; i++){
	sz_1.x = (cu[i].x*cu[i].x + cu[i].y*cu[i].y) - (cl[i].x*cl[i].x + cl[i].y*cl[i].y); 
	sz_1.y = 0.; 
	sm_1.x = cu[i].x*cl[i].x + cu[i].y*cl[i].y;
	sm_1.y = -cu[i].x*cl[i].y + cu[i].y*cl[i].x; 
	sp_1.x = cu[i].x*cl[i].x + cu[i].y*cl[i].y;
	sp_1.y = cu[i].x*cl[i].y - cu[i].y*cl[i].x;
	
	sz[i].x = sz_1.x; sz[i].y = sz_1.y;
	sm[i].x = sm_1.x; sm[i].y = sm_1.y; 
	
	a_sp[i].x = 0.; a_sp[i].y = 0.;
	a_sz[i].x = 0.; a_sz[i].y = 0.;
	a_sm[i].x = 0.; a_sm[i].y = 0.; 
	
	for (int j = 0; j < num_ens; j++){
		sz_2.x = (cu[j].x*cu[j].x + cu[j].y*cu[j].y) - (cl[j].x*cl[j].x + cl[j].y*cl[j].y); 
		sz_2.y = 0.; 
		sm_2.x = cu[j].x*cl[j].x + cu[j].y*cl[j].y;
		sm_2.y = -cu[j].x*cl[j].y + cu[j].y*cl[j].x; 
		
		sm_sp[j + i*num_ens].x = sm_2.x*sp_1.x - sm_2.y*sp_1.y; 
		sm_sp[j + i*num_ens].y = sm_2.x*sp_1.y + sm_2.y*sp_1.x; 
		
		sm_sz[j + i*num_ens].x = sm_2.x*sz_1.x - sm_2.y*sz_1.y;
		sm_sz[j + i*num_ens].y = sm_2.x*sz_1.y + sm_2.y*sz_1.x;
		
		sm_sm[j + i*num_ens].x = sm_2.x*sm_1.x - sm_2.y*sm_1.y;
		sm_sm[j + i*num_ens].y = sm_2.x*sm_1.y + sm_2.y*sm_1.x;
				
		sz_sz[j + i*num_ens].x = sz_2.x*sz_1.x - sz_2.y*sz_1.y;
		sz_sz[j + i*num_ens].y = sz_2.x*sz_1.y + sz_2.y*sz_1.x;	
	}
}

// on GUP side 
double2 *ap_a_dev,*a_dev,*a_a_dev;
double2 *sz_dev,*sm_dev,*a_sz_dev,*a_sm_dev,*a_sp_dev;
double2 *sm_sp_dev,*sm_sz_dev,*sm_sm_dev,*sz_sz_dev;

// create observables on GPU side 
cudaMalloc((void**)&ap_a_dev,sizeof(double2));
cudaMalloc((void**)&a_dev,sizeof(double2));
cudaMalloc((void**)&a_a_dev,sizeof(double2));

cudaMalloc((void**)&sz_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&sm_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&a_sz_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&a_sm_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&a_sp_dev,num_ens*sizeof(double2));

cudaMalloc((void**)&sm_sp_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&sm_sz_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&sm_sm_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&sz_sz_dev,num_ens*num_ens*sizeof(double2));



// copy observables on GPU side 
cudaMemcpy(ap_a_dev,&ap_a,sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(a_dev,&a,sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(a_a_dev,&a_a,sizeof(double2),cudaMemcpyHostToDevice);

cudaMemcpy(sz_dev,sz,num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(sm_dev,sm,num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(a_sz_dev,a_sz,num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(a_sm_dev,a_sm,num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(a_sp_dev,a_sp,num_ens*sizeof(double2),cudaMemcpyHostToDevice);

cudaMemcpy(sm_sp_dev,sm_sp,num_ens*num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(sm_sz_dev,sm_sz,num_ens*num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(sm_sm_dev,sm_sm,num_ens*num_ens*sizeof(double2),cudaMemcpyHostToDevice);
cudaMemcpy(sz_sz_dev,sz_sz,num_ens*num_ens*sizeof(double2),cudaMemcpyHostToDevice);

//***************
// derivatives 
double2 *d_ap_a_dev,*d_a_dev,*d_a_a_dev;
double2 *d_sz_dev,*d_sm_dev,*d_a_sz_dev,*d_a_sm_dev,*d_a_sp_dev;
double2 *d_sm_sp_dev,*d_sm_sz_dev,*d_sm_sm_dev,*d_sz_sz_dev;

// create observables on GPU side 
cudaMalloc((void**)&d_ap_a_dev,sizeof(double2));
cudaMalloc((void**)&d_a_dev,sizeof(double2));
cudaMalloc((void**)&d_a_a_dev,sizeof(double2));

cudaMalloc((void**)&d_sz_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&d_sm_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&d_a_sz_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&d_a_sm_dev,num_ens*sizeof(double2));
cudaMalloc((void**)&d_a_sp_dev,num_ens*sizeof(double2));

cudaMalloc((void**)&d_sm_sp_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&d_sm_sz_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&d_sm_sm_dev,num_ens*num_ens*sizeof(double2));
cudaMalloc((void**)&d_sz_sz_dev,num_ens*num_ens*sizeof(double2));

FILE *Result_time,*Result_Sz,*Result_Sm,*Result_photon,*Result_field, *coherences_real, *coherences_imag;
// time of simulation
Result_time = fopen("Result_time.dat","w");
Result_Sz = fopen("Result_Sz.dat","w");
Result_photon = fopen("Result_photon.dat","w");
Result_field = fopen("Result_field.dat","w");
Result_Sm = fopen("Result_Sm.dat","w");
coherences_real = fopen("coherences_real.dat","w");







// ***********************************
// simulations starts
// ***********************************
double tc;


// update the old reduced density matrix 
for (int t = 1; t < t_num; t++){

	tc = t*t_step;
	//printf("%1f \n", tc);

//************************************
// calculate derivatives 

// calculate the photon observables
// ap_a, a, a_a 
	calculate_photons<<<1,1>>>(tc,num_ens,para_a_dev,para_c_dev,\
				ap_a_dev,a_dev,a_a_dev,\
				a_sp_dev,sm_dev,a_sm_dev,\
				d_ap_a_dev,d_a_dev,d_a_a_dev);
	cudaThreadSynchronize();

// calculate the atomic observables and atom-photon correlations
// sz, sm, a_sz, a_sm, a_sp 
	calculate_atoms<<<1,num_ens>>>(tc,num_ens,para_a_dev,para_c_dev,\
					sz_dev,sm_dev,a_sz_dev,a_sm_dev,a_sp_dev,\
					sm_sp_dev,sm_sm_dev,sm_sz_dev,a_dev,ap_a_dev,a_a_dev,\
					d_sz_dev,d_sm_dev,d_a_sz_dev,d_a_sm_dev,d_a_sp_dev);
	cudaThreadSynchronize();

// calculate the atom-atom correlations 
// sm_sp, sm_sz, sm_sm, sz_sz
	calculate_correlations<<<num_ens,num_ens>>>(num_ens,t_step,para_a_dev,para_c_dev,\
						sm_sp_dev,sm_sz_dev,sm_sm_dev,sz_sz_dev,\
						a_dev,a_sm_dev,a_sp_dev,a_sz_dev,sm_dev,sz_dev,\
						d_sm_sp_dev,d_sm_sz_dev,d_sm_sm_dev,d_sz_sz_dev);
	cudaThreadSynchronize();

//*************************************
// update observables

	update_photons<<<1,1>>>(t_step,ap_a_dev,a_dev,a_a_dev,\
				d_ap_a_dev,d_a_dev,d_a_a_dev);
	cudaThreadSynchronize();


	update_atoms<<<1,num_ens>>>(num_ens,t_step,para_a_dev,sz_dev,sm_dev,a_sz_dev,a_sm_dev,a_sp_dev,\
				d_sz_dev,d_sm_dev,d_a_sz_dev,d_a_sm_dev,d_a_sp_dev);
	cudaThreadSynchronize();
	
	update_correlations<<<num_ens,num_ens>>>(num_ens,t_step,sm_sp_dev,sm_sz_dev,sm_sm_dev,sz_sz_dev,\
						d_sm_sp_dev,d_sm_sz_dev,d_sm_sm_dev,d_sz_sz_dev);
	cudaThreadSynchronize();

	if ( t%t_store == 0) {
	// copy the calculate observables back to CPU side 


		cudaMemcpy(sz,sz_dev,num_ens*sizeof(double2),cudaMemcpyDeviceToHost);
		cudaMemcpy(sm,sm_dev,num_ens*sizeof(double2),cudaMemcpyDeviceToHost);
		cudaMemcpy(sm_sp,sm_sp_dev,num_ens*sizeof(double2),cudaMemcpyDeviceToHost);
		cudaMemcpy(&ap_a,ap_a_dev,sizeof(double2),cudaMemcpyDeviceToHost);
		cudaMemcpy(&a,a_dev,sizeof(double2),cudaMemcpyDeviceToHost);
		cudaThreadSynchronize();



	// store the file
		fprintf(Result_time,"%e \n",(double)t*t_step);
		fprintf(Result_photon,"%e \n",ap_a.x);
		fprintf(Result_field,"%e %e \n",a.x,a.y);
		//printf("%1f	%e	\n",tc, ap_a.x);
	
	/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

		for (int i = 0; i < num_ens; i++) {
			fprintf(Result_Sz,"%e ",sz[i].x);
			fprintf(Result_Sm,"%e %e  ",sm[i].x, sm[i].y);
			fprintf(coherences_real,"%e ",sm_sp[i].x);
		}
		fprintf(Result_Sz,"\n");
		fprintf(Result_Sm,"\n");
		fprintf(coherences_real,"\n");
		
	//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
	}
}


// close the files
fclose(Result_time);
fclose(Result_Sz);
fclose(Result_photon);
fclose(Result_field);
fclose(coherences_real);


// close the memories 

cudaFree(para_a_dev); cudaFree(para_c_dev); cudaFree(t_step_dev);

cudaFree(ap_a_dev); cudaFree(a_dev); cudaFree(a_a_dev);

cudaFree(sz_dev); cudaFree(sm_dev); cudaFree(a_sz_dev);
cudaFree(a_sm_dev);cudaFree(a_sp_dev);

cudaFree(sm_sp_dev); cudaFree(sm_sz_dev);
cudaFree(sm_sm_dev); cudaFree(sz_sz_dev);

cudaFree(d_ap_a_dev); cudaFree(d_a_dev); cudaFree(d_a_a_dev);

cudaFree(d_sz_dev); cudaFree(d_sm_dev); cudaFree(d_a_sz_dev);
cudaFree(d_a_sm_dev);cudaFree(d_a_sp_dev);

cudaFree(d_sm_sp_dev); cudaFree(d_sm_sz_dev);
cudaFree(d_sm_sm_dev); cudaFree(d_sz_sz_dev);



ct1=clock();
fprintf(stderr,"Program takes about %.2f s\n",(double)(ct1-ct0)/(double)CLOCKS_PER_SEC);
return 0;
}

In [24]:

%%cmd bash
set PATH=%PATH%;"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.35.32215\bin\Hostx64\x64"
echo %PATH%
nvcc -w functions.cu main.cu -o file 

Microsoft Windows [Version 10.0.19042.2604]
(c) Microsoft Corporation. All rights reserved.

(c:\Users\Admin\Documents\Sam\CUDA_code\.conda) c:\Users\Admin\Documents\Sam\CUDA_code>set PATH=%PATH%;"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.35.32215\bin\Hostx64\x64"

(c:\Users\Admin\Documents\Sam\CUDA_code\.conda) c:\Users\Admin\Documents\Sam\CUDA_code>echo %PATH%
c:\Users\Admin\Documents\Sam\CUDA_code\.conda;c:\Users\Admin\Documents\Sam\CUDA_code\.conda\Library\mingw-w64\bin;c:\Users\Admin\Documents\Sam\CUDA_code\.conda\Library\usr\bin;c:\Users\Admin\Documents\Sam\CUDA_code\.conda\Library\bin;c:\Users\Admin\Documents\Sam\CUDA_code\.conda\Scripts;c:\Users\Admin\Documents\Sam\CUDA_code\.conda\bin;C:\Users\Admin\anaconda3\condabin;c:\Users\Admin\.vscode-cli\server-stable\bin\441438abd1ac652551dbe4d408dfcec8a499b8bf\bin\remote-cli;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.0\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.0\libnvvp;C:\Pr

main.cu(2): fatal error C1083: Cannot open include file: 'unistd.h': No such file or directory


main.cu

(c:\Users\Admin\Documents\Sam\CUDA_code\.conda) c:\Users\Admin\Documents\Sam\CUDA_code>

In [ ]:
!nvcc --version

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin